In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

در این نوت بوک فقط تمرکز رو داده قیمت خونه هست.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy import stats

In [ ]:
df=pd.read_csv('../input/housepriceprediction/train_data.csv')

In [ ]:
df.head()

In [ ]:
df['SalePrice'].describe()

In [ ]:
sns.set_style('whitegrid')

def three_chart_plot(df, feature):

    fig = plt.figure(constrained_layout = True, figsize = (12, 8))
    grid = gridspec.GridSpec(ncols = 3, nrows = 3, figure = fig)
    
    ax1 = fig.add_subplot(grid[0, :2])
    ax1.set_title('Histogram')
    
    sns.distplot(df.loc[:, feature], norm_hist = True, ax = ax1)
    plt.axvline(x = df[feature].mean(), c = 'red')
    plt.axvline(x = df[feature].median(), c = 'green')
    
    ax2 = fig.add_subplot(grid[1, :2])
    ax2.set_title('QQ_plot')
    stats.probplot(df.loc[:,feature], plot = ax2)

    ## Customizing the Box Plot. 
    ax3 = fig.add_subplot(grid[:, 2])

    ## Set title. 
    ax3.set_title('Box Plot')
    sns.boxplot(df.loc[:,feature], ax = ax3 ,orient = 'v')

In [ ]:
three_chart_plot(df, 'SalePrice');

1. histogram
 1. خط قرمزمیانگ و خط سبز میانه هست 
 1. همینطور که میبینید نمودار هیستوگرام چولگی دارد

1. boxplot
 1. نمودار نشان میده که داده پرت داریم

1. probability plot
 1.  خط قرمز نشان میده که داد ه های متغییر پاسخ اگه روی این خط باشند توزیع نرمال دارند.
 1.  نمودار آبی مقادیر واقعی رو نشان میدهند. 

In [ ]:
from scipy.stats import skew
print("Skewness of the SalesPrice is", df['SalePrice'].skew())

In [ ]:
np.mean(df['SalePrice'])

In [ ]:
category_var = df.select_dtypes(include = 'object')
num_var = df.select_dtypes(exclude = 'object')

print("Number of categorical features are: ", category_var.shape[1])
print("Number of numerical features are: ", num_var.shape[1])


In [ ]:
num = df.select_dtypes(exclude = 'object')
corr_num = num.corr()

f, ax = plt.subplots(figsize = (25,1 ))

sns.heatmap(corr_num.sort_values(by = ['SalePrice'], ascending = False).head(1))

plt.title('Correlation of Numerical Features with the Target', weight = 'bold', fontsize = 18)
plt.xticks(weight='bold')
plt.yticks(weight='bold', color='cyan', rotation=0)

plt.show()

همبستگی که بین داده ها و متغییر پاسخ هست رو نشان میدهد.

In [ ]:
correlation = df.corr()['SalePrice'].sort_values(ascending = False).to_frame().head(10)

cmap = sns.light_palette("cyan", as_cmap = True)

s = correlation.style.background_gradient(cmap = cmap)

s


همانطور که میبینیم کیفیفت در قیمت خونه تاثیر دارد و در زندگی عادی هم همینطور هست.
محیط زندگی هم با افزایش محیط باعث بالارفتن قیمت خونه میشه

From the above table, we can conclude that better the overall quality of the house, more will be the price of the house which is also true in real life

In [ ]:
plt.figure(figsize = ((15, 6)))
plt.scatter(x = 'GrLivArea', y = 'SalePrice', data = df, color='crimson', alpha=0.4)
plt.title('Relationship between Living Area and Sale Price', weight = 'bold', fontsize = 10)
plt.xlabel('Area of the house', weight = 'bold')
plt.ylabel('Sale Price', weight = 'bold')
plt.show()

a linear relationship between Living Area and the Sale Price.
If the area is huge then price should also have also been high but the two rightmost points suggest something else. These two points are outliers. We will drop both of them below.

همانطور که میبینید رابطه خطی دارند و داده های بیشتر از 4500 میتونه داده پرت باشه که ما حذف میکنیم.

Remove area of house >4400

In [ ]:
df= df[df['GrLivArea'] < 4400]

plt.figure(figsize = ((15, 6)))
plt.scatter(x = df['GrLivArea'], y = 'SalePrice', data = df, color='crimson', alpha=0.4)
plt.title('Relationship between Living Area and Sale Price', weight = 'bold', fontsize = 10)
plt.xlabel('Area of the house', weight = 'bold')
plt.ylabel('Sale Price', weight = 'bold')
plt.show()


We can observe that those 3 values have been successfully dropped.
سه تا از داده ها حذف شدند

In [ ]:
plt.figure(figsize = ((15, 6)))
plt.scatter(x = 'GarageArea', y = 'SalePrice', data = df, color='crimson', alpha=0.4)
plt.title('Relationship between Garage Area and Sale Price', weight = 'bold', fontsize = 10)
plt.xlabel('Area of the Garage', weight = 'bold')
plt.ylabel('Sale Price', weight = 'bold')
plt.show()

گاراژهم رابطه خطی دارد 

In [ ]:
plt.figure(figsize = ((15, 6)))
plt.scatter(x = 'TotalBsmtSF', y = 'SalePrice', data = df, color = 'crimson', alpha = 0.4)
plt.title('Relationship between Total Basement Area and Sale Price', weight = 'bold', fontsize = 10)
plt.xlabel('Total Basement Area (in Square Feets)', weight = 'bold')
plt.ylabel('Sale Price', weight = 'bold')
plt.show()


این نمودار همم رابطه خطی دارد.

In [ ]:
 
fig, (ax1, ax2) = plt.subplots(figsize = (12, 8), ncols = 2,sharey = False)

sns.scatterplot(x = df['GrLivArea'], y = df['SalePrice'],  ax = ax1)
## Putting a regression line. 
sns.regplot(x = df['GrLivArea'], y = df['SalePrice'], ax = ax1,color ='red')

sns.scatterplot(x = df['MasVnrArea'],y = df['SalePrice'], ax = ax2)
## regression line for MasVnrArea and SalePrice. 
sns.regplot(x = df['MasVnrArea'], y = df['SalePrice'], ax=ax2)

در نمودار قرمز رابطه بسیار خطی بین متغییرها نسبت به نمودار آبی دارد.
پس بصورت جدا میایم نمودار باقی مانده برای دو متغییر خطی بالا رو چک میکنیم.

# saleprice and GrlivArea are much more linear

In [ ]:
plt.subplots(figsize = (12,8))

sns.residplot(df['GrLivArea'], df['SalePrice'])

 در حالت ایده آل اگر مفروضات برقرار باشند باقی مانده ها به صورت تصادفی در اطراف خط صفر بدون هیچ الگویی پراکنده میشوند.
 نمودار در اینجا شبیه قیف هست.
 با افزایش مقدار متغییر واریانس زیاد میشودکه این ویژگی روHeteroscedasticity گویند.
 

 the residual plot looks more like a funnel.
 The error plot shows that as GrLivArea value increases, the variance also increases, which is the characteristics known as Heteroscedasticity. 

Now let's make sure that the target variable follows a normal distribution.

In [ ]:
df["SalePrice_log"] = np.log(df["SalePrice"])
# df = train
three_chart_plot(df, 'SalePrice_log')

در اینجا نمودار هیستوگرام به صورت زنگوله شد 
qq-plot هم با خط قرمز همپوشانی دارد.


We can observe that the SalesPrice feature now follows a normal distibution as the Histogram resembles bell-shape

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize = (15, 6), ncols = 2, sharey = False,  sharex = False)

sns.residplot(df['GrLivArea'], df['SalePrice'], ax = ax1).set_title('SalePrice')
sns.residplot(df['GrLivArea'], df['SalePrice_log'], ax = ax2).set_title('SalePrice_log')

همانطور که میبینم نمودار از حالت قیف خارج شده و پس ویژگی Homoscedastic رو دارد.


We can see that the scatterplot on the left had heteroscedasticity(funnel like shape) but after applying log transformation to the feature, it was removed and now it's Homoscedastic.

In [ ]:
df["SalePrice_sqrt"] = np.sqrt(df["SalePrice"])
# df = train
three_chart_plot(df, 'SalePrice_sqrt')

 در اینجا تغییر نداشتیم پس این تابع رو قبول نمیکنیم

In [ ]:
plt.figure(figsize = ((25, 10)))
df.boxplot(by = 'OverallQual', column = 'SalePrice');


As the quality increases, price of the houses also increase
با افزایش کیفیت ، قیمت خانه ها نیز افزایش می یابد

In [ ]:
def horizontal_plot(data, column_name, color, plot_title):
    
    """
    A function that plots a horizontal bar graph for the given column name
    """
    
    fig, ax = plt.subplots(figsize = (9, 6))
    
    title_cnt = data[column_name].value_counts().sort_values(ascending = False).reset_index()

    mn = ax.barh(title_cnt.iloc[:, 0], title_cnt.iloc[:, 1], edgecolor = 'black', color =
                sns.color_palette(color, len(title_cnt)))

    for s in ['top','bottom','left','right']:
        ax.spines[s].set_visible(False)

    # Remove x,y Ticks
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')

    # Add padding between axes and labels
    ax.xaxis.set_tick_params(pad=5)
    ax.yaxis.set_tick_params(pad=10)

    # Add x,y gridlines
    ax.grid(b=True, color='grey', linestyle='-.', linewidth=1, alpha=0.2)

    # Show top values 
    ax.invert_yaxis()

    ax.set_title(plot_title,weight='bold',loc='center', pad=10, fontsize=16)
    
    ax.set_xlabel('Count', weight='bold')

    for i in ax.patches:
        ax.text(i.get_width() + 1.0, i.get_y() + 0.5, str(round((i.get_width()), 2)),
                fontsize=10, fontweight='bold', color='grey')

    plt.yticks(weight='bold')
    plt.xticks(weight='bold')

    plt.show()

In [ ]:
horizontal_plot(df, 'Neighborhood', 'Reds', 'Most frequent neighborhoods')


In [ ]:
horizontal_plot(df, 'BldgType', 'Greens', 'Building type: Type of dwelling')

1. نوع خونه ها Type of dwelling
1. 1. حدود 1099 مربوط به تک خانواده است و
1. 1. 27 تا مربوط به تک خانواده هس ولی دو خانواده زندگی میکنن.

In [ ]:
horizontal_plot(df, 'GarageCond', 'Blues', 'Garage Condition')

Most of the houses have Typically Average Garage Condition. Only couple of houses have Excellent Garage Condition.

In [ ]:
horizontal_plot(df, 'GarageType', 'terrain', 'Garage Type')

Most of the houses have Attached Garages and only 17 have Garages in the Basement.